In [1]:
import json
from tqdm import tqdm
from copy import deepcopy

In [8]:
split = 'test'
dataset = 'aol'
file_in = '/mnt/scratch/chenqu/{}/original/session_{}.txt'.format(dataset, split)
file_out = '/mnt/scratch/chenqu/{}/preprocessed/session_{}.txt'.format(dataset, split)

if dataset == 'aol':
    CLICKS = 'clicks'
    TITLE = 'title'
elif dataset == 'msmarco':
    CLICKS = 'candidates'
    TITLE = 'content'
    
file_out = '/mnt/scratch/chenqu/aol_new/preprocessed/session_{}.txt'.format(split)

In [9]:
def create_examples(line, dataset):
        
        # session_info = [turn_1: [query, clicked_title, skipped_title],
        #                 turn_2: [query, clicked_title, skipped_title],
        #                 ...
        #                 turn_n: [query, clicked_title, skipped_title],
        #                ]
        
        session_info = []
        doc_num_list = []
        
        session = json.loads(line.strip())
        session_id = session['session_id']
        session_len = len(session['query'])
        
        # if we do not include skipped, each turn has a query, a clicked title,
        # if include skipped, each turn has another component, which is the skipped title
        num_components_per_turn = 3
        
        # session_examples: [history_component_1, history_component_2, ...]
        session_examples = []
        
        
        query_id = 0
        for turn_id, turn in enumerate(session['query']):
            
            clicks_list = [click['label'] for click in turn[CLICKS]]
            total_clicks = sum(clicks_list)
            
            query = turn['text']
            
            clicked_indices = [i for i, x in enumerate(clicks_list) if x]
            clicked_title = turn[CLICKS][clicked_indices[0]][TITLE] if len(clicked_indices) > 0 else '[EMPTY_TITLE]'
            max_skip_index = clicked_indices[-1] + 1 if len(clicked_indices) > 0 else 0
                        
            try:
                skipped_index = clicks_list.index(False)
                if skipped_index <= max_skip_index:
                    skipped_title = turn[CLICKS][skipped_index][TITLE]
                else:
                    skipped_title = '[EMPTY_TITLE]'
            except:
                skipped_title = '[EMPTY_TITLE]'
            
            session_info.append([query, clicked_title, skipped_title])            
            
            if (total_clicks > 0 and turn_id == session_len - 1) or dataset == 'aol':
                doc_id = 0
                for doc_id, doc in enumerate(turn[CLICKS]):
                    title = doc[TITLE]
                    label = doc['label']
                    history = deepcopy(session_info[:-1])
                    guid = '{}_{}_{}'.format(session_id, query_id, doc_id)
                    session_examples.append({'guid': guid, 'query': query, 'title': title, 'label': label, 'history': history})
                    
                    doc_id += 1
                    
            query_id += 1
        return session_examples

In [10]:
with open(file_in) as fin, open(file_out, 'w') as fout:
    for line in tqdm(fin):
        session_examples = create_examples(line, dataset)
        for example in session_examples:
            fout.write(json.dumps(example, sort_keys=True) + '\n')

29369it [02:10, 225.49it/s]
